In [2]:
import os
from meld_classifier.meld_cohort import MeldCohort, MeldSubject
import numpy as np
import pandas as pd


In [4]:
#initialise dataset

cohort = MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6.hdf5',
               dataset='MELD_dataset_V6.csv')

# initialise subjects manually in list or get from a csv dataset
subject_ids, trainval_ids, test_ids  = cohort.read_subject_ids_from_dataset()



In [8]:
len(trainval_ids)

472

In [52]:
import pandas as pd

#parameters for graph

df=pd.DataFrame()
values={}
for si,subj in enumerate(trainval_ids):
    values['ID']=subj
    s = MeldSubject(subj,cohort=cohort)
    values['age']=s.get_demographic_features('Age at preop')
    values['group']=s.group
    values['MRI negative']=s.get_demographic_features('Ever reported MRI negative')
    values['histopathology']=s.get_demographic_features('Histology')
    values['seizure_free'] = s.get_demographic_features('Seizure free')
    df=pd.concat([df,pd.DataFrame([values])])

In [53]:
df.groupby(['group','seizure_free']).count()

ID  age  MRI negative  histopathology
group   seizure_free                                        
patient 0.0            61   61            61              58
        1.0           124  122           124             116

In [56]:
df

,ID,age,group,MRI negative,histopathology,seizure_free
0,MELD2_H7_3T_FCD_003,10.0,patient,0.0,FCD_2A,0.0
0,MELD2_H7_3T_FCD_006,12.0,patient,0.0,FCD_2B,1.0
0,MELD2_H7_3T_FCD_007,22.0,patient,1.0,FCD_3,0.0
0,MELD2_H7_3T_FCD_008,4.0,patient,1.0,FCD_3,0.0
0,MELD2_H7_3T_FCD_010,0.5,patient,0.0,FCD_2B,1.0
...,...,...,...,...,...,...
0,MELD_H9_3T_FCD_0007,21.0,patient,0.0,FCD_2A,1.0
0,MELD_H9_3T_FCD_0008,38.0,patient,0.0,NaN,NaN
0,MELD_H9_3T_FCD_0009,30.0,patient,1.0,NaN,NaN
0,MELD_H9_3T_FCD_0011,25.0,patient,1.0,NaN,NaN


In [60]:
exclude_subjects_training = df[(df['group']=='patient') & (df['seizure_free']!=1)]['ID'].values
print(len(exclude_subjects_training))
exclude_subjects_training

168


array(['MELD2_H7_3T_FCD_003', 'MELD2_H7_3T_FCD_007',
       'MELD2_H7_3T_FCD_008', 'MELD2_H7_3T_FCD_013',
       'MELD_H10_3T_FCD_0001', 'MELD_H10_3T_FCD_0003',
       'MELD_H10_3T_FCD_0004', 'MELD_H10_3T_FCD_0006',
       'MELD_H10_3T_FCD_0011', 'MELD_H10_3T_FCD_0015',
       'MELD_H12_15T_FCD_0002', 'MELD_H12_3T_FCD_0004',
       'MELD_H12_3T_FCD_0010', 'MELD_H12_3T_FCD_0019',
       'MELD_H12_3T_FCD_0020', 'MELD_H12_3T_FCD_0021',
       'MELD_H12_3T_FCD_0022', 'MELD_H12_3T_FCD_0025',
       'MELD_H12_3T_FCD_0026', 'MELD_H12_3T_FCD_0027',
       'MELD_H14_3T_FCD_0005', 'MELD_H14_3T_FCD_0007',
       'MELD_H14_3T_FCD_0012', 'MELD_H14_3T_FCD_0015',
       'MELD_H14_3T_FCD_0018', 'MELD_H14_3T_FCD_0027',
       'MELD_H14_3T_FCD_0033', 'MELD_H14_3T_FCD_0036',
       'MELD_H14_3T_FCD_0038', 'MELD_H14_3T_FCD_0041',
       'MELD_H15_3T_FCD_0001', 'MELD_H15_3T_FCD_0006',
       'MELD_H15_3T_FCD_0009', 'MELD_H15_3T_FCD_0011',
       'MELD_H16_3T_FCD_002', 'MELD_H16_3T_FCD_020',
       'MELD_H1

In [61]:
## change training and test dataset

csv = '/rds/project/kw350/rds-kw350-meld/meld_data/Data/MELD_dataset_V6.csv'
df_new = pd.read_csv(csv)

In [62]:
df_new

,Unnamed: 0,subject_id,split
0,0,MELD2_H7_3T_FCD_001,test
1,1,MELD2_H7_3T_FCD_002,test
2,2,MELD2_H7_3T_FCD_003,trainval
3,3,MELD2_H7_3T_FCD_004,test
4,4,MELD2_H7_3T_FCD_005,test
...,...,...,...
940,940,MELD_H9_3T_FCD_0009,trainval
941,941,MELD_H9_3T_FCD_0010,test
942,942,MELD_H9_3T_FCD_0011,trainval
943,943,MELD_H5_3T_FCD_0031,trainval


In [63]:
# for row in df_new.iterrows:
df_new[(df_new['split']=='trainval')&(df_new['subject_id'].isin(exclude_subjects_training))]

,Unnamed: 0,subject_id,split
2,2,MELD2_H7_3T_FCD_003,trainval
6,6,MELD2_H7_3T_FCD_007,trainval
7,7,MELD2_H7_3T_FCD_008,trainval
12,12,MELD2_H7_3T_FCD_013,trainval
45,45,MELD_H10_3T_FCD_0001,trainval
...,...,...,...
927,927,MELD_H6_3T_FCD_0020,trainval
939,939,MELD_H9_3T_FCD_0008,trainval
940,940,MELD_H9_3T_FCD_0009,trainval
942,942,MELD_H9_3T_FCD_0011,trainval


In [64]:
# for row in df_new.iterrows:
df_new.loc[(df_new['split']=='trainval')&(df_new['subject_id'].isin(exclude_subjects_training)), 'split'] = 'test'

In [77]:
df_new.drop(df_new.columns[df_new.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [78]:
df_new

,subject_id,split
0,MELD2_H7_3T_FCD_001,test
1,MELD2_H7_3T_FCD_002,test
2,MELD2_H7_3T_FCD_003,test
3,MELD2_H7_3T_FCD_004,test
4,MELD2_H7_3T_FCD_005,test
...,...,...
940,MELD_H9_3T_FCD_0009,test
941,MELD_H9_3T_FCD_0010,test
942,MELD_H9_3T_FCD_0011,test
943,MELD_H5_3T_FCD_0031,test


In [79]:
df_new.to_csv('/rds/project/kw350/rds-kw350-meld/meld_data/Data/MELD_dataset_V6_seizurefree.csv')